In [73]:
import json
import requests
import pandas as pd

from datetime import datetime
from user_agent import generate_user_agent

In [36]:
headers = {
    "User-Agent": generate_user_agent(),
    "Authorization": open("resy_key.txt").readlines(),
}

In [18]:
# Resy does not give a list of restaurants in "nyc", just those in a radius of a point.
# Use a bunch of points so we do not miss any restaurants.
points = (
    (40.712941, -74.006393), # Downtown
    (40.731248, -73.995177), # West Village
    (40.727896, -73.983021), # East Village
    (40.758192, -73.982890), # Midtown
    (40.785470, -73.976007), # Upper West
    (40.778061, -73.954034), # Upper East
    (40.810670, -73.946289), # Harlem
    (40.845711, -73.937796), # Washington Heights
    (40.816246, -73.918893), # South Bronx
    (40.769370, -73.911535), # Astoria
    (40.749186, -73.893048), # Jackson Heights
    (40.759380, -73.832923), # Flushing
    (40.710153, -73.960520), # Williamsburg
    (40.703422, -73.987805), # Dumbo
    (40.684090, -73.977800), # Downtown Bk
)

In [56]:
slug_to_info = {}

for lat, lng in points:
    response = requests.post("https://api.resy.com/3/venuesearch/search", headers=headers, json={
        "page": 1,
        "per_page": 1000,
        "geo": {"latitude": lat, "longitude": lng, "radius": 35420},
        "query": ""
    })
    
    # Get basic info about the restaurants.
    for hit in json.loads(response.text)["search"]["hits"]:
        slug_to_info[hit["url_slug"]] = {
            "slug": hit["url_slug"],
            "id": hit["id"]["resy"],
            "cuisine": hit["cuisine"][0],
            "stars": hit["rating"]["average"],
            "count": hit["rating"]["count"],
            "where": hit["neighborhood"],
        }

In [84]:
url = "".join([
    "https://api.resy.com/4/venue/calendar",
    "?venue_id={}",
    "&num_seats=2",
    "&start_date=" + datetime.today().strftime("%Y-%m-%d"),
    "&end_date=2030-01-01"
])

# Get reservation info about the restaurants.
for slug, info in slug_to_info.items():
    try:
        response = requests.get(url.format(info["id"]), headers=headers)
        schedule = json.loads(response.text)["scheduled"]
        info["available"] = sum(1 for s in schedule if s["inventory"]["reservation"] == "available")
        info["sold-out"] = sum(1 for s in schedule if s["inventory"]["reservation"] == "sold-out")
    except:
        pass

In [92]:
for slug, info in slug_to_info.items():
    try:
        del info["booked"]
    except:
        pass

In [93]:
df = pd.DataFrame.from_records(list(slug_to_info.values()))
df.to_csv("restaurants.csv", index=False)

In [94]:
df

,slug,id,cuisine,stars,count,where,available,sold-out
0,high-tide,70590,American,5.0,25,Dumbo,120,0
1,pilot-ny,69720,Seafood,5.0,20,Brooklyn Heights,120,0
2,foxface-natural,71434,New American,5.0,16,East Village,0,0
3,fragile-flour,66716,Dessert,5.0,6,East Village,5,0
4,4-charles-prime-rib,834,Steakhouse,4.93,20744,West Village,0,19
...,...,...,...,...,...,...,...,...
1923,emily-squared-park-slope,67562,Pizza,3.96,241,Brooklyn,7,0
1924,hendrix-house-bedstuy,65200,Caribbean,3.63,86,Bedstuy,105,0
1925,king-mother,50851,Natural Wine,4.94,422,Ditmas Park,14,0
1926,sweet-catch-brooklyn,68665,Seafood,4.61,106,Prospect Lefferts Gardens,78,0
